# IIC-3800 Tópicos en CC - NLP UC

- Versiones de librerías, python 3.8.10

- numpy 1.20.3
- nltk 3.7
- lime 0.2.0.1
- spacy 3.5.1
- gcsfs 2023.3.0
- protobuf 3.20.3

In [1]:
import pandas as pd

df = pd.read_csv("film_affinity.csv")
df

,critica,nota,url
0,"Bueno, bajo mi gusto, otro fracaso más de DC. ...",3,https://www.filmaffinity.com/es/reviews/1/4208...
1,Es tan terrible que podría funcionar como paro...,1,https://www.filmaffinity.com/es/reviews/1/4208...
2,Tengo una tradición desde hace más de 5 años. ...,2,https://www.filmaffinity.com/es/reviews/1/4208...
3,No entiendo como nadie tiene la cara de presen...,1,https://www.filmaffinity.com/es/reviews/1/4208...
4,La primera entrega de Wonder Woman (2017) no m...,4,https://www.filmaffinity.com/es/reviews/1/4208...
...,...,...,...
4795,"""Shrek"" es sin lugar a dudas una de las mejore...",9,https://www.filmaffinity.com/es/reviews/6/4945...
4796,"Muy buena e incluso diría, inteligente comedia...",8,https://www.filmaffinity.com/es/reviews/3/9420...
4797,Cuando una película consigue hacer que algo ta...,7,https://www.filmaffinity.com/es/reviews/3/9420...
4798,Una gran comedia estupida que cumple su funció...,8,https://www.filmaffinity.com/es/reviews/3/9420...


____________________________________________________________________________________________________________

## Actividad en clase

Construya un clasificador de sentimiento en **castellano** usando el dataset FilmAffinity y Spacy. Para esto haga lo siguiente:

- Cree la columna 'sentiment' a partir de **nota**. Sentiment debe considerar dos clases: **positivo** y **negativo**. Observe que nota va de 1 a 10. 
- Preprocese el texto del campo **critica** usando lo que hemos visto en clases. Note que deberá cambiar algunos procesos ya que el texto está en **castellano**.
- Particione el dataset en particiones de training/development/testing (75/15/10)
- Convierta los datos para procesarlos con Spacy (use **convert**). 
- Entrene un clasificador reproduciendo el método visto en clases.
- Evalúe el clasificador sobre la particion de test.
- Cuanto termine, me avisa para entregarle una **L (logrado)**.
- Recuerde que las L otorgan un bono en la nota final de la asignatura.


***Tiene hasta el final de la clase.***

_________________________________________________________________________________________________________________

# Solución

!python3 -m spacy download es_core_news_sm

In [28]:
import string
import re
import spacy
from spacy.lang.es.stop_words import STOP_WORDS

nlp = spacy.load("es_core_news_sm") # a spanish-based nlp model
REGX_USERNAME = r"@[A-Za-z0-9$-_@.&+]+"

def preprocessing(text):
  text = text.lower()
  text = re.sub(REGX_USERNAME, ' ', text)
  tokens = [token.text for token in nlp(text)]
  tokens = [t for t in tokens if t not in STOP_WORDS and t not in string.punctuation and len(t) > 2]
  tokens = [t for t in tokens if not t.isdigit()]

  return " ".join(tokens)


df["text_clean"] = df["critica"].apply(preprocessing)

In [30]:
df["sentiment"] = df.apply(lambda x: 0 if int(x["nota"]) < 5  else 1, axis=1)

In [31]:
df.head()

,critica,nota,url,text_clean,sentiment
0,"Bueno, bajo mi gusto, otro fracaso más de DC. ...",3,https://www.filmaffinity.com/es/reviews/1/4208...,gusto fracaso empezó año aves presa acaba año ...,0
1,Es tan terrible que podría funcionar como paro...,1,https://www.filmaffinity.com/es/reviews/1/4208...,terrible funcionar parodia sobreactuada record...,0
2,Tengo una tradición desde hace más de 5 años. ...,2,https://www.filmaffinity.com/es/reviews/1/4208...,tradición años diciembre cine blockbuster peli...,0
3,No entiendo como nadie tiene la cara de presen...,1,https://www.filmaffinity.com/es/reviews/1/4208...,entiendo cara presentar película entiendo crít...,0
4,La primera entrega de Wonder Woman (2017) no m...,4,https://www.filmaffinity.com/es/reviews/1/4208...,entrega wonder woman pareció maravilla contrar...,0


In [32]:
dataset = list(df[["text_clean", "sentiment"]].sample(frac=1).itertuples(index=False, name=None))
train_data = dataset[:3600]  # 75%
dev_data = dataset[3600:4320] # 15%
test_data = dataset[4320:] # 10%

In [36]:
def convert(data, outfile):
    db = spacy.tokens.DocBin()
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        doc.cats["POS"] = label == 1
        doc.cats["NEG"] = label == 0
        db.add(doc)
    
    db.to_disk(outfile)
convert(train_data, "./train.spacy")
convert(dev_data, "./dev.spacy")
convert(test_data, "./test.spacy")

In [37]:
!python3 -m spacy init config --lang es --pipeline textcat --optimize efficiency --force config.cfg

2023-03-16 14:21:34.825555: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-16 14:21:35.336576: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-16 14:21:35.336634: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-16 14:21:35.336641: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-16 14:21:36.272648: W tensorf

In [38]:
!python3 -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy --output model --verbose

2023-03-16 14:21:38.678015: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-16 14:21:39.199346: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-16 14:21:39.199408: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-16 14:21:39.199415: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-16 14:21:40.119332: W tensorf

In [39]:
!python3 -m spacy evaluate ./model/model-best/ ./test.spacy

2023-03-16 14:22:26.125458: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-16 14:22:26.636211: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-16 14:22:26.636268: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-16 14:22:26.636274: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-16 14:22:27.560119: W tensorf

In [40]:
texts = ["La pelicula es innecesariamente larga. A ratos se vuelve aburrida y es dificil de seguir.", "Me arrepiento de haber comprado en esta tienda."]
nlp = spacy.load("./model/model-best")
for text in texts:
    doc = nlp(preprocessing(text))
    print(doc.cats,  "-",  text)

{'POS': 0.1581939309835434, 'NEG': 0.8418060541152954} - La pelicula es innecesariamente larga. A ratos se vuelve aburrida y es dificil de seguir.
{'POS': 0.4453706741333008, 'NEG': 0.5546293258666992} - Me arrepiento de haber comprado en esta tienda.
